# Simulator

In [2]:
import sys
sys.path.insert(0, "..")
import pandas as pd
import matplotlib.pyplot as plt
from trading_simulator import TradingSimulator, DLSimulator
from dl_solutions.dlsolutions import CryptoDLSolutions
from data_processor.data_processing import DataProcessor
from utils.crypto_utils import ModelAnalysis
from keras import optimizers

Using TensorFlow backend.


## Classic

### Data preparation

In [ ]:
cryptos = ['ADA', 'BTC', 'ETH', 'LTC', 'LNK']
sel = 0

In [ ]:
for crypto in cryptos:
    processor = DataProcessor([crypto])
    sim = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2, log = False)
    sim.simulate()

In [ ]:
processor = DataProcessor([crypto])
sim1 = TradingSimulator(processor, crypto, strategy = [1], loss_allowed = 0.2)
sim2 = TradingSimulator(processor, crypto, strategy = [2], loss_allowed = 0.2)
sim3 = TradingSimulator(processor, crypto, strategy = [3], loss_allowed = 0.2)
sim4 = TradingSimulator(processor, crypto, strategy = [4], loss_allowed = 0.2)
sim5 = TradingSimulator(processor, crypto, strategy = [5], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [6], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [7], loss_allowed = 0.2)
sim7 = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2)

### Simulations

In [ ]:
sim1.simulate()

In [ ]:
sim2.simulate()

In [ ]:
sim3.simulate()

In [ ]:
sim4.simulate()

In [ ]:
sim5.simulate()

In [ ]:
sim6.simulate()

In [ ]:
sim7.simulate()

## DL Solution

In [3]:
model_analysis = ModelAnalysis()

def execute_test(split_size = -1):
    '''
    Creates artificial test process to validate different hyperparameters
    '''
    sim = DLSimulator(crypto, prev_periods, pred_periods, columns, target,
    norm_strat, model_sel, layers, neurons, batch_size, epochs, 
    activation, loss, metrics, optimizer, initial_learning_rate, callbacks)

    df_train = sim.get_df().iloc[:split_size]
    if split_size == -1:
        df_test = sim.get_df().iloc[split_size:]
    else:
        df_test = sim.get_df().iloc[split_size:split_size+1]
    
    display(df_train)
    display(df_test)

    sim.train_model(df_train)
    
    pred = sim.predict(pd.DataFrame(df_test))
    print('Predictions', pred)
    h = sim.get_history()
    model_analysis.draw_history(h)

### Data preparation

Estos son los parametros ademas de otros pocos añadidos ad hoc al modelo (dropout). Estos parametros iran cambiando a lo largo del analisis.

En este notebook no aparece todo el analisis por ser demasiado extenso (analisis por dias de la semana, otras metricas, cpas, cambios en la output layer, uso de previous y pred periods...)

In [ ]:
crypto = 'ETH'

#### PERIODS ####
prev_periods = 20
pred_periods = 20


#### COLUMNS ####
columns_1 = ['RSI', 'close']

#### HYPERPARAMETERS ####
layers = 4
neurons = [25, 25, 25, 25, 25] # El primer valor son las neuronas de la base layer
batch_size = 64
epochs = 100
#activations = ['relu', 'sigmoid', 'softmax']
#losses = ['mse', 'binary_crossentropy', 'categorical_crossentropy']
#metrics_opt = ['mse', 'accuracy']
activation = 'relu'
loss = 'mse'
metrics = ['mse']
optimizer = 'adam'
initial_learning_rate = 0.01
callbacks = ['mc', 'es']

### DL parameters and training

Creation of different strategies:

- Based on result
- Based on close/indicators normalized
- Try with different close distances
- Multioutput
- Based on multiple operation (softmax)

#### LSTM 

In [ ]:
model_sel = 0

##### One line feature

Usaremos solo una característica como objetivo de predicción

Análisis por periodos fijos:

Analizaremos el impacot de los heprparámetros para posteriormente buscar los valores optimos de periodos previos y a predecir

Fijaremos los periodos usados y a predecir en 20



Cambiaremos lo siguiente en cuanto a hiperparámetros y modelo sobre dos metricas usadas: una métrica normalizada como RSI y otro sin normalizar como close:

- Layers
- Neurons
- Epochs
- Tipo de normalización
- Activacion y función de pérdidas

Probemos primero diferentes tipos de noramalización con la siguientes caracterisiticas del modelo:

Normalizacion:
- 0: Normalizar sobre todo el dataset
- 1: Min Max Scaller (No valida al ser una sola line de test y por tanto normalizar con respecto a nada)
- 2: Min Max por columnas sobre todo el dataset
- 3: Ln
- 4: Sin normalización


In [ ]:
target = 'close'

In [ ]:
columns = columns_1
num_features = len(columns)

In [ ]:
norm_strat = 0
print('Normalizacion:', norm_strat)
execute_test()

In [ ]:
norm_strat = 2
print('Normalizacion:', norm_strat)
execute_test()

In [ ]:
norm_strat = 3
print('Normalizacion:', norm_strat)
execute_test()

In [ ]:
norm_strat = 4
print('Normalizacion:', norm_strat)
execute_test()

Mejor metodo de normalizacion es el 2

Nos centramos ahora en la seleccion de columnas. Queremos la mayor información posible. Ahora en vez de selccionar solo una columna como target, seleccionaremos varias para proveer a nuestro futuro bucle de prediccion e inversion de mas información.

Para esto, lo mejor será tener la información del precio (suavizada o sin suavizar con SMA o EMA) y algun indicador del momento como RSI, ADX o SO.

##### Multiples features

Usaremos multiples características como objetivos de predicción. De esta manera nuestro modelo final proveera mayor información.

Usaremos indicadores ya normalizados entre ciertos valores como RSI, ADX y SO. Entre estos tres ADX suele tener mejores capacidades a la hora de indicar la fuerza de la tendencia. Utilizaremos ADX combinado con algun valor del precio:
- close
- SMA
- EMA

Esta vez cambiaremos entre disferentes monedas para dar más validez al resultado final

In [ ]:
cryptos = ['ETH', 'ADA', 'BTC', 'LNK', 'LTC']

In [ ]:
columns_2 = ['ADX', 'close']
columns_3 = ['ADX', 'EMA_50']
columns_4 = ['ADX', 'SMA 50']
columns_5 = ['ADX', 'EMA_200']
columns_6 = ['ADX', 'SMA 200']

target = None
norm_strat = 2

close

In [ ]:
columns = columns_2
num_features = len(columns)
for crypto in cryptos:
    crypto = crypto
    execute_test()

EMA 50

In [ ]:
columns = columns_3
num_features = len(columns)
for crypto in cryptos:
    crypto = crypto
    execute_test()

SMA 50

In [ ]:
columns = columns_4
num_features = len(columns)
for crypto in cryptos:
    crypto = crypto
    execute_test()

EMA 200

In [ ]:
columns = columns_5
num_features = len(columns)
for crypto in cryptos:
    crypto = crypto
    execute_test()

SMA 200

In [ ]:
columns = columns_6
num_features = len(columns)
for crypto in cryptos:
    crypto = crypto
    execute_test()

Tras analizar los resultados obtenidos observamos ligeras mejores cuando el valor del precio esta suavizado (menos outliers). Entre SMA y EMA parece que EMA se comporta ligeramente mejor. No es algo determinante pero seguiremos usando EMA porque cambia de forma mas rapida y de acuerda al valor real de las tendencias porque tiene mas en cuenta los ultimos precios a la hora de calcularse

Además hemos visto dificultades en la red para predecir el valor de la métrica ADX. De ahora en adelante analizaremos RSI en vez de ADX pero es posible que si no conseguimos buenas predicciones usemos solo el valor del precio, sus diferencias o la operación mas adecuada (se comenta más adelante estos dos nuevos métodos)

Antes de seguir probaremos dos metodos mas:
- Uso de la diferencia de precio
- Uso de la operación mas conveniente

In [ ]:
columns_7 = ['RSI', 'close_diff_5']
columns_8 = ['RSI', 'close_diff_10']
columns_9 = ['RSI', 'close_diff_20']
columns_11 = ['op_buy', 'op_sell', 'op_hold'] # Usar softmax

Dado que actualmente tenemos puesto el parametro de dias de prediccion a 20, usaremos la columna con un calculo de diferencia de precio de 20 dias -> columns_9

In [ ]:
columns = columns_9
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Este metodo ofrece resultados aceptables en comparación con el calculo del precio como valor y por tanto usaremos este en vez del precio final como prediccion dado que si calculamos el precio final luego debemos calcular cuanto mas sube o baja con respecto al dia actual. Este método ya nos da directamente ese valor

Como podemos ver la precision de prediccion no es demasiado acertada para las metricas y por tanto evitaremos su uso en nuestro modelo final

La precision de este metodo mejorará al usar menos dias de predicción. De todas formas trataremos de mejorar la red con 20 dias en adelante antes de bajar a 10 o 5 dias. Además es posible que no exista una relación temparal tal como para el precio. Por eso podemos encontrar dificultades en el modelo para generar buenos resultados.

En adelante usaremos la combinacion de precio con diferencia de precio. Si al modelo le costara mucho predecir la diferencia de precio usaremos solo el precio.

----------------------

Para calcular la operacion mas adecuada debemos cambiar la red para aceptar una activación softmax

Vamos a buscar ejemplos donde se realiza una venta o compra en la fila de test para poner mas aprueba al modelo (debido a que la mayoria son hold).

Debemos añadir un class weight en el modelo. Por eso analizamos el peso de cada clase

In [ ]:
columns = columns_11

activation = 'softmax'
loss = 'categorical_crossentropy'
metrics = ['categorical_crossentropy', 'accuracy']

crypto = 'ETH'

sim = DLSimulator(crypto, prev_periods, pred_periods, columns, target,
    norm_strat, model_sel, layers, neurons, batch_size, epochs, 
    activation, loss, metrics, optimizer, initial_learning_rate, callbacks)

df = sim.get_df()

print(len(df[(df['op_buy_0']==1)]))
print(len(df[(df['op_sell_0']==1)]))
print(len(df[(df['op_hold_0']==1)]))

Añadiremos al modelo un x4 para las clases sell y buy

The index [-7, -9, -15, -19] provide examples with buy and sell

In [ ]:
split_indexes = [-7, -9, -15, -19]

for split_index in split_indexes:
    execute_test(split_index)

En los cuatro casos el modelo no ha acertado. Consideraremos esta opción para nuestro modelo final una vez optimizado sobre el precio

----------

El siguiente análisis se centrará en los hiperparametros y modelado de la red (layers, epochs, batch size, dropout). Antes de comenzar inicializaremos los parametros que dejaremos fijos durante esta etapa de analisis 

In [ ]:
#### PERIODS ####
prev_periods = 20
pred_periods = 20

#LSTM
model_sel = 0 

# Cryptos
cryptos = ['ETH', 'ADA', 'BTC', 'LNK', 'LTC']

#### NORM AND FEATURES CONFIGURATION ####
target = None
norm_strat = 2

#### COLUMNS ####
columns_11 = ['op_buy', 'op_sell', 'op_hold'] # Usar softmax
columns_12 = ['close', 'close_diff_20']


#### Hyper params ####

#activations = ['relu', 'sigmoid', 'softmax']
#losses = ['mse', 'binary_crossentropy', 'categorical_crossentropy']
#metrics_opt = ['mse', 'accuracy']
activation = 'relu'
loss = 'mse'
metrics = ['mse']
optimizer = 'adam'
initial_learning_rate = 0.01
callbacks = ['mc', 'es']
batch_size = 64
epochs = 100

Modelado:

Modelaremos la red en las capas LSTM

Modifcamos el numero de capas:

- 1 Capas
- 4 Capas
- 10 Capas

Y neuronas

- 10 Nueronas
- 50 Neuronas
- 100 Neuronas
- Gradual invertido

Trabajaremos sobre diferencia de precio de 20 dias

In [ ]:
layers = 1
neurons = [25, 25]

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Los resultados son incluso mejores para algunos sets que con la configuración inicial.

Iremos subiendo capas y neuronas hasta que perdamos eficiencia en nuestros resultados

In [ ]:
layers = 1
neurons = [100, 100]

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Añadir neuronas no ha mejorado resultados en todos los casos pero si en unos pocos en el test, consiguiendo buenos datos en el set de validación pero no mejores que antes. Probaremos añadiendo capas y dejando las neuronas mas bajas. Dejaremos la capa base con mas neuronas.

El error en BTC puede deberse a un estancamiento por un lr muy alto. Quizás sin callbacks o con lr mas bajo hubieramos alcanzado un minimo.

In [ ]:
layers = 2
neurons = [100, 20, 20]

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Vemos mejoras, aunque la diferencia de precio no esta obteniendo buenas predicciones en la fila de test. Como hemos comentado es posible que no tenga una relacion secuencial tan alta como el precio.

Sin embargo seguimos viendo como las funciones de perdida son más bajas que con el modelo usado como base. Esto es algo positivo.

Probaremos ahora a bajar capas de nuevo y subir neuronas

In [ ]:
layers = 1
neurons = [200, 100]

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Vemos que tenemos algo de overfitting en esta prueba. Sin embargo los datos de perdidas son bastante buenos. Seguiremos con esta idea e intentaremos paliar el overfitting más adelante.

In [ ]:
layers = 2
neurons = [200, 100, 50]

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Definitivamente añadiendo capas nuestro modelo se hace mucho más complejo de manejar y obtiene valores de perdidas peores. Sin embargo los test realzados son mejores. Probaremos una vez más a subir el numero de nueronas con solo una capa oculta. Posiblemente esto genere más overfitting que seremos capaces de resolver más adelante.

Tengamos en cuenta que si a este modelo le quitamos o añadimos paciencia al early stopping junto con un lr mas bajo y epocas mas altas, posiblemente obtengamos buenos resultados.

In [ ]:
layers = 1
neurons = [400, 200]

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Bajaremos el numero de nueronas y lo igualamos a la capa oculta. El earlystopping esta impidiendo que el modelo encuentre minimos a tiempo

In [ ]:
layers = 1
neurons = [300, 300]

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Nos quedaremos con dos capas ocultas y 300 + 150 + 75 neuronas. Probemos sin early stop porque nos esta impidiendo llegar a minimos.

In [ ]:
layers = 2
neurons = [300, 150, 75]

callbacks = ['mc']

columns = columns_12
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Hemos obtenido una buena configuración con:

2 capas ocultas

300 base

150 + 75 en capas ocultas

Como columnas seguiremos usando el precio como tal dado que los resultados han sido buenos. Si encontraramos dificultades podriamos volver a usar EMA.


Añadimos/quitamos ad hoc en el modelo capas de dropout y su activación

Tenemos algo de overfitting en nuestro modelo que ya contiene 0.2 de dropout. Se supone que deberíamos subirlo algo y ver si mejora. En caso de aumento de perdidas probaremos a reducirlo de nuevo

In [ ]:
#### PERIODS ####
prev_periods = 20
pred_periods = 20

#LSTM
model_sel = 0 

# Cryptos
cryptos = ['ETH', 'ADA', 'BTC', 'LNK', 'LTC']

#### NORM AND FEATURES CONFIGURATION ####
target = None
norm_strat = 2

#### COLUMNS ####
columns_11 = ['op_buy', 'op_sell', 'op_hold'] # Usar softmax
columns_12 = ['close', 'close_diff_20']


#### Hyper params ####

#activations = ['relu', 'sigmoid', 'softmax']
#losses = ['mse', 'binary_crossentropy', 'categorical_crossentropy']
#metrics_opt = ['mse', 'accuracy']
activation = 'relu'
loss = 'mse'
metrics = ['mse']
initial_learning_rate = 0.01

optimizer = 'adam'

callbacks = ['mc', 'es']
batch_size = 64
epochs = 100

layers = 2
neurons = [300, 150, 75]

columns = columns_12
num_features = len(columns)

Dropout 0.3

In [ ]:
for crypto in cryptos:
    crypto = crypto
    execute_test()

No observamos mejoras

Probamos con Dropout 0.1

Eliminaremos el overfitting con regularizers en el kernel l1_l2 si mejora con un dropout mas bajo

In [ ]:
for crypto in cryptos:
    crypto = crypto
    execute_test()

No vemos mejoras. Nos quedaremos con Dropout 0.2

Vemos que el learning rate puede estar algo alto pues alcanzamos minimos muy pronto y los set de validación no terminan de fijarse en un valor de perdidas (acaban dando pequeños saltos). Probaremos a bajarlo y a eliminar de nuevo el early stop

Se han hechos pruebas con el lr shceduler sin exito

In [ ]:
callbacks = ['mc']

learning_rate=0.0005

In [ ]:
optimizer = optimizers.Adam(learning_rate=0.0005)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Eliminaremos el overfitting que podamos tener con el uso de regularizacion del kernel l1_l2

kernel_regularizer=tensorflow.keras.regularizers.l1_l2(l1=0.01, l2=0.01)

Aprovechamos para volver a probar una sola capa

In [ ]:
#### PERIODS ####
prev_periods = 20
pred_periods = 20

#LSTM
model_sel = 0 

# Cryptos
cryptos = ['ETH', 'ADA', 'BTC', 'LNK', 'LTC']

#### NORM AND FEATURES CONFIGURATION ####
target = None
norm_strat = 2

#### COLUMNS ####
columns_11 = ['op_buy', 'op_sell', 'op_hold'] # Usar softmax
columns_12 = ['close', 'close_diff_20']


#### Hyper params ####

#activations = ['relu', 'sigmoid', 'softmax']
#losses = ['mse', 'binary_crossentropy', 'categorical_crossentropy']
#metrics_opt = ['mse', 'accuracy']
activation = 'relu'
loss = 'mse'
metrics = ['mse']
initial_learning_rate = 0.01

optimizer = optimizers.Adam(learning_rate=0.0005)

callbacks = ['mc', 'es']
batch_size = 64
epochs = 100

layers = 1
neurons = [300, 150]

columns = columns_12
num_features = len(columns)

Se ha probado regularizar en recurrent layer y bias pero los resultados no son optimos

El siguiente resultado aplica solo a kernel

Para reducir el contenido no se muestran los resultados aplicados a recurrent y bias. Asi como varias pruebas con regularizers mas altos que tampoco han dado buenos resultados

In [ ]:
for crypto in cryptos:
    crypto = crypto
    execute_test()

No hemos conseguido mejorar el modelo con regularización

Ahora modificaremos el batch size y epocas

Si bien, pareciera que 100 epocas han sido más que suficiente y no tocaremos este parametro a no ser que veamos falta de convergencia al modificar el batch

Hemos probado con batch size de 64. Probemos con un batch mas pequeño para actualizar de forma mas recurrente los pesos. Si mejora seguiremos reduciendo

Batch size 32

In [ ]:
batch_size = 32

for crypto in cryptos:
    crypto = crypto
    execute_test()

Batch size 16

El entrenamiento se hace muy lento y por ello bajaremos el numeor de epochs

In [ ]:
epochs = 70

batch_size = 16

for crypto in cryptos:
    crypto = crypto
    execute_test()

Batch size 8

In [ ]:
epochs = 40

batch_size = 8

for crypto in cryptos:
    crypto = crypto
    execute_test()

Nos quedaremos con un bacth size de 16. Observamos ligeras mejoras. Con 8 se descontrola algo el aprendizaje. Se vuelve muy lento. Posiblemente se vuelva a porbar mas adelante una vez seteados los demas parametros

Probaremos otras activaciones y funcion de perdidas

msle podria funcionar porque los movimientos del mercado son proporcionales al volumen de inversion y precios de la moneda

In [ ]:
epochs = 70
batch_size = 16

In [ ]:
activation = 'relu'
loss = 'msle'
metrics = ['msle']

for crypto in cryptos:
    crypto = crypto
    execute_test()

La distribucion de precios se asemeja a una distribucion log normal (right skewed). Por eso esta funcion de periddas funciona mejor

Puede perjudicar a close_diff porque no sigue la misma distribución

De todas formas no dejaremos de lado mse.

In [ ]:
activation = 'relu'
loss = 'mae'
metrics = ['mae']

for crypto in cryptos:
    crypto = crypto
    execute_test()

Obtenemos mejores resultados con la funcion de perdidas: msle

Probaremos ahora diferentes activaciones

- Leaky ReLU para evitar dying neurons
- Swish ha demostrado buen comportamiento según algunos papers

In [ ]:
loss = 'msle'
metrics = ['msle']

epochs = 70

In [ ]:
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from keras.layers import Activation
from keras.layers import LeakyReLU

def custom_activation(x, beta = 1):
        return (K.sigmoid(beta * x) * x)

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

In [ ]:
activation = LeakyReLU(alpha=0.05)

for crypto in cryptos:
    crypto = crypto
    execute_test()

In [ ]:
activation = Activation(custom_activation,name = "Swish")

for crypto in cryptos:
    crypto = crypto
    execute_test()

Nos quedaremos con activación ReLU

In [ ]:
activation = 'relu'

Finalmente usaremos la diferencia de precio y el precio pero aumentando a 40 los dias previos usados y bajando a 10 los dias a predecir

In [ ]:
#### PERIODS ####
prev_periods = 40
pred_periods = 10


columns_12 = ['close', 'close_diff_10']
columns = columns_12

for crypto in cryptos:
    crypto = crypto
    execute_test()

Mejora considerable al subir prev peridos y bajar pred periods

Se ha probado a mover el numero de dias previos usados. Parece no afectar mucho al comportamiento del modelo 

Haremos backtesting para probar a añadir varias caracteristicas y sacar como target la diferencia de precio y el precio. Obtendremos mejor resultado una vez aplicado el modelado de red y los hiperparametros mas eficiantes segun hemos analizado?

Caracteristicas a probar:

- Resistance and support levels 50

Target a probar:

- close
- close_diff_10

Cambiaremos a mse porque no vimos mejoras relevantes en la diferencia de precio usando msle.

Se ha probado en este modelado de datos con mse y no ofrece buenos resultados. Se vuelve a mse

In [ ]:
loss = 'mse'
metrics = ['mse']

In [ ]:
target = 'close'

columns_13 = ['Sup 50', 'Res 50', 'close']
columns_14 = ['Sup 50', 'Res 50', 'close_diff_10']

columns = columns_13
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

In [ ]:
target = 'close_diff_10'

columns = columns_14
num_features = len(columns)

for crypto in cryptos:
    crypto = crypto
    execute_test()

Seguiremos usando el modelado anterior

Por ultimo, antes de probar el modelo GRU haremos una ultima prueba con softmax y la operacion a realizar usando los parametros y modelado ya analizado

In [ ]:
epochs = 70

target = None
columns_11 = ['op_buy', 'op_sell', 'op_hold'] # Usar softmax

columns = columns_11
num_features = len(columns)

activation = 'softmax'
loss = 'categorical_crossentropy'
metrics = ['accuracy', 'categorical_crossentropy']

crypto = 'ETH'

split_indexes = [-7, -9, -15, -19]

for split_index in split_indexes:
    execute_test(split_index)

No conseguimos buenos resultados. Seguimos sin ser capaces de identificar los puntos de compra y venta

#### GRU

Probaremos el mejor modelado e hiperparametros ya vistos pero con GRU

In [ ]:
#### PERIODS ####
prev_periods = 40
pred_periods = 10

#GRU
model_sel = 1

# Cryptos
cryptos = ['ETH', 'ADA', 'BTC', 'LNK', 'LTC']

#### NORM AND FEATURES CONFIGURATION ####
target = None
norm_strat = 2

#### COLUMNS ####
columns_12 = ['close', 'close_diff_10']


#### Hyper params ####

#activations = ['relu', 'sigmoid', 'softmax']
#losses = ['mse', 'binary_crossentropy', 'categorical_crossentropy']
#metrics_opt = ['mse', 'accuracy']
activation = 'relu'
loss = 'msle'
metrics = ['msle']
initial_learning_rate = 0.01

optimizer = optimizers.Adam(learning_rate=0.0005)

callbacks = ['mc', 'es']
batch_size = 16
epochs = 70

layers = 1
neurons = [300, 150]

columns = columns_12
num_features = len(columns)

In [ ]:
for crypto in cryptos:
    crypto = crypto
    execute_test()

#### Final models to use

Despues de todo el analisis usaremos este diccionario de parametros para nuestros modelos. Bajamos aun mas los dias a predecir y los dias previos usados para mayor precisión y rapidez de entrenamiento.

Se ha observaod ademas estancamientos en algunos entrenamiento y para evitarlo hemos cambiado de msle a mse

Se ha probado con diferentes funciones de lr scheduler por no solucionaba el problema.

Otra solucion es obtener mas datos

In [ ]:
#### PERIODS ####
prev_periods = 1
pred_periods = 5

#LSTM
model_sel = 0

# Cryptos
cryptos = ['ETH_H', 'ADA', 'BTC', 'LNK', 'LTC']

#### NORM AND FEATURES CONFIGURATION ####
target = None
norm_strat = 2

#### Hyper params ####
activation = 'relu'
loss = 'msle'
metrics = ['msle']
initial_learning_rate = 0.01

optimizer = optimizers.Adam(learning_rate=0.0005)

callbacks = ['mc', 'es']
batch_size = 16
epochs = 70

layers = 1
neurons = [300, 150]

columns = ['close']
num_features = len(columns)

In [ ]:
for crypto in cryptos:
    crypto = crypto
    execute_test()

Hemos probado con un dataset mas grande y vemos una precisión considerablemente más alta

## Complete simulation

Complete strategy definition. Let the machine work through all the dataset

In [4]:
#### PERIODS ####
prev_periods = 1
pred_periods = 5

#LSTM
model_sel = 0

# Cryptos
crypto = 'BTC'

#### NORM AND FEATURES CONFIGURATION ####
target = None
norm_strat = 2

#### Hyper params ####
activation = 'relu'
loss = 'msle'
metrics = ['msle']
initial_learning_rate = 0.01

optimizer = optimizers.Adam(learning_rate=0.0005)

callbacks = ['mc', 'es']
batch_size = 16
epochs = 70

layers = 1
neurons = [300, 150]

columns = ['close']
num_features = len(columns)

In [5]:
sim = DLSimulator(crypto, prev_periods, pred_periods, columns, target,
    norm_strat, model_sel, layers, neurons, batch_size, epochs, 
    activation, loss, metrics, optimizer, initial_learning_rate, callbacks)

Loading... BTC
Extracting columns columns for BTC
Proccessing and arranging columns for LSTM model


In [6]:
sim.simulate()

Simulation starting...
INDEX: 1118
CLOSE: 10373.44
MODE: buy
TRAINING MODEL:
TRAIN LAST ROWS:        close_0     close
1237  31988.71  40088.22
1238  33949.53  38150.02
1239  36769.36  35404.47
1240  39432.28  34051.24
1241  40582.81  37371.38
Input shape: (1118, 1, 1) (1118, 1)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 300)            362400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               270600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151       
Total params: 633,151
Trainable params: 633,151
Non-trainable params: 0
________________________________